In [13]:
from ibpythonic import ibConnection, message
from ibapi.contract import Contract

from time import sleep, strftime
from datetime import datetime

import pandas as pd
import numpy as np

In [14]:
def makeStkContract(contractTuple):
    newContract = Contract()
    newContract.symbol = contractTuple[0]
    newContract.secType = contractTuple[1]
    newContract.exchange = contractTuple[2]
    newContract.currency = contractTuple[3]
    return newContract

In [15]:
bar_data_list = []
busy = False
active_symbol = 0

In [16]:
def my_hist_data_handler(msg):
    global bar_data_list
    # print('Receiving bar: ' + datetime.now().strftime("%H:%M:%S.%f"))
    bar_data_list.append(msg.bar)

In [17]:
def my_hist_data_end_handler(msg):
    global bar_data_list
    global busy
    global active_symbol
        
    # print('Creating df: ' + datetime.now().strftime("%H:%M:%S.%f"))
    df = pd.DataFrame(columns=('date', 'open', 'high', 'low', 'close', 'volume'))
    for bar in bar_data_list:
        row = {'date': bar.date, 'open': bar.open, 'high': bar.high, 'low': bar.low, 'close': bar.close, 'volume': bar.volume}
        df = df.append(row, ignore_index=True)
    df.loc[:, 'date'] = pd.to_datetime(df.date, format='%Y%m%d') #'%Y%m%d  %H:%M:%S'
    df = df.set_index('date')

    # print('Writing CSV: ' + datetime.now().strftime("%H:%M:%S.%f"))
    sy = symbols_df.loc[active_symbol, 'Symbol']
    ex = symbols_df.loc[active_symbol, 'Exchange']
    df.to_csv('data/' + sy + '_' + ex + '.csv')
    
    # write finished info to symbol download list
    symbols_df.loc[active_symbol, 'Status'] = 'Finished'
    
    bar_data_list = []
    active_symbol = []
    busy = False

In [18]:
def my_error_handler(msg):
    global bar_data_list
    global busy
    global active_symbol
    
    # write msg info to symbol download list
    symbols_df.loc[active_symbol, 'Status'] = msg
    
    bar_data_list = []
    active_symbol = []
    busy = False

In [23]:
file = open('symbollist.txt')
symbols = file.read().splitlines()
file.close()
symbols = symbols[:3]

symbols_df = pd.read_csv('symbol_database.csv', sep=';')
symbols_df['Status'] = 'Untouched'
symbols_df = symbols_df.loc[1:20, :]

# symbols = ['DAI', 'BMW', 'SAP', 'BAY']
# symbols = ['TSLA', 'ADBE', 'NVDA', 'AMD', 'INTL']

endtime = strftime('%Y%m%d %H:%M:%S')

In [24]:
con = ibConnection(port=7497)
con.register(my_hist_data_handler, message.historicalData)
con.register(my_hist_data_end_handler, message.historicalDataEnd)
con.register(my_error_handler, message.error)

True

In [25]:
con.connect()
    
for index, row in symbols_df.iterrows():
    global busy

    # print('Creating contract: ' + datetime.now().strftime("%H:%M:%S.%f"))
    contractTuple = (row['Symbol'], 'STK', row['Exchange'], row['Currency'])
    stkContract = makeStkContract(contractTuple)

    active_symbol = index
    # print('Requesting data: ' + datetime.now().strftime("%H:%M:%S.%f"))
    busy = True
    con.reqHistoricalData(1,stkContract,endtime,"10 Y","1 day","MIDPOINT",1,1, keepUpToDate=False, chartOptions=[])
    
    while busy is True:
        sleep(0.5)
    # print('Finished symbol: ' + datetime.now().strftime("%H:%M:%S.%f"))
    sleep(1)

con.disconnect()

unhandled exception in EReader thread
Traceback (most recent call last):
  File "c:\users\marti\dev\_pyvenvs\latest36\lib\site-packages\ibapi\reader.py", line 34, in run
    data = self.conn.recvMsg()
  File "c:\users\marti\dev\_pyvenvs\latest36\lib\site-packages\ibapi\connection.py", line 99, in recvMsg
    buf = self._recvAllMsg()
  File "c:\users\marti\dev\_pyvenvs\latest36\lib\site-packages\ibapi\connection.py", line 119, in _recvAllMsg
    buf = self.socket.recv(4096)
OSError: [WinError 10038] An operation was attempted on something that is not a socket


True

In [26]:
symbols_df

,Unnamed: 0,Currency,Exchange,Name,Symbol,Symbol2,Status
1,1,EUR,FWB,Lyxor Euro Corporate Bond Ex Financials UCITS ETF,CNB,LYBF,"<error id=-1, errorCode=2104, errorMsg=Market ..."
2,2,EUR,FWB,UBS ETF - MSCI Emerging Markets Socially Respo...,UEF5,UEF5,"<error id=1, errorCode=162, errorMsg=Historica..."
3,3,EUR,FWB,BNP Paribas Easy Barclays Euro Government Infl...,GOMA,GOMA,"<error id=1, errorCode=162, errorMsg=Historica..."
4,4,EUR,FWB,Coba ETC -2 x Platinum Daily Short Index,X0BM,X0BM,Finished
5,5,EUR,FWB,Lyxor Barclays Floating Rate Euro 0-7Y UCITS ETF,FLOT,NK4L,"<error id=1, errorCode=162, errorMsg=Historica..."
6,6,EUR,FWB,Coba ETC 2x Gasoil Daily Long Index,X0E2,X0E2,Finished
7,7,EUR,FWB,L&G Ecommerce Logistics UCITS ETF,ETLH,ETLH,"<error id=1, errorCode=162, errorMsg=Historica..."
8,8,EUR,FWB,ComStage ETF PSI 20 Leverage UCITS ETF,PP2,CD48,"<error id=1, errorCode=162, errorMsg=Historica..."
9,9,EUR,FWB,Xtrackers II Italy Government Aggregate Bond S...,XBO3,XBO3,"<error id=1, errorCode=162, errorMsg=Historica..."
10,10,EUR,FWB,WisdomTree Japan Equity UCITS ETF,WTIZ,WTIZ,"<error id=1, errorCode=162, errorMsg=Historica..."


In [ ]:
 con.disconnect()

# Enhancements

### Multiple symbols downloading
- Just pass a list of symbols

### Speedup
- dict of symbols with ascending numbers
- download data one after another, symbol-number is id for downloading
- write received data to file as fast as possible, filename is symbol-number
- download next symbol as soon as data is in file
- in parallel load finished data from files, convert to dataframes and save as csv

In [ ]:
with open('your_file.txt', 'w') as f:
    for item in my_list:
        f.write("%s\n" % item)

# or

outfile.write("\n".join(itemlist))

# or

import pickle

with open('outfile', 'wb') as fp:
    pickle.dump(itemlist, fp)
    
with open ('outfile', 'rb') as fp:
    itemlist = pickle.load(fp)
    
# or

open("bla.txt", "wb").write(''.join(random.choice(string.ascii_lowercase) for i in xrange(10**7)))

### Extra features for downloading
- check for existing data
- start time for downloading is end time of data. last candle of data is considered invalid
- check for splits/mergers

### Symbol in multiple exchanges
- if symbol is multiple times in list
    - load data from primary exchange if ppossible
    - if primary is not available, load from exchange with highest average (price-)volume


### Symbol database
- pull symbols from ib webpage
- create symbol database
- pull symbol data from ib
- abillity to create own univestes (=symbol lists) based on symbol data

### Storage of price data
- store prica data in CSV per symbol per timframe
- store price data in database? sqlite, as handeled in one file?
- cloud storage of data

### Misc
- implement error handler